<a href="https://colab.research.google.com/github/ishumann/Coding-Implementation/blob/main/Keras_tuner_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import seaborn as sns

In [ ]:
df= pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Keras-Tuner/main/Real_Combine.csv')


In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units= hp.Int('units_'+str(i),
                                             min_value=32,
                                             max_value=512,
                                             step = 32),
                               activation= "relu"))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer = keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2,1e-3,1e-4])),
        loss='mean_absolute_error', metrics=['mean_absolute_error'])
    return model



In [ ]:
tuner = RandomSearch(
    build_model,
    objective = 'val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name = "air Quality index")

In [ ]:
tuner.search_space_summary()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=32, test_size=.3 )

In [ ]:
tuner.search(X_train,
             y_train,
             epochs=5,
             validation_data=(X_test,y_test))

In [ ]:
tuner.results_summary()

In [ ]:

def build_model(hp):
    model = keras.Sequential()
    for _ in range(hp.Int('num_layers', 2, 5)):  # Reduce max layers for stability
        model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=256, step=32),
                                activation="relu",
                                kernel_initializer="he_normal"))  # He initialization
    model.add(layers.Dense(1, activation="linear"))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
        loss="mean_absolute_error",
        metrics=["mean_absolute_error"],
        clipnorm=1.0  # Gradient clipping
    )
    return model

class EarlyStoppingWithNaN(keras.callbacks.Callback):
  def on_train_batch_end(self, batch, logs=None):
    if logs is None:
      return
    for metric, value in logs.items():
      if np.isnan(value):
        print(f"Detected NaN value in metric: {metric}. Stopping training.")
        self.model.stop_training = True
        break

early_stopping_nan = EarlyStoppingWithNaN()

tuner = RandomSearch(
    build_model,
    objective="val_mean_absolute_error",
    max_trials=15,  # Increase trials for more exploration
    executions_per_trial=1,  # Reduce executions per trial for efficiency
    directory="project",
    project_name="air_quality_index"
)

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
